In [5]:
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import wandb
from transformers import ViTMAEConfig

from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
from transformers.models.vit_mae.modeling_vit_mae import ViTMAEModel
# from tem_dataloader import MultimodalDatasetPerTrajectory
from torch.utils.data import DataLoader

from d3rlpy.algos import CQL
from d3rlpy.dataset import Episode, MDPDataset, Transition
wandb.login() 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yadix. Use `wandb login --relogin` to force relogin


True

In [8]:
from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
import torch
# call CustomViT
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig
from PIL import Image

# output_dir='/home/tyz/Desktop/11_777/camelmera/weights'
trained_model_name = 'multimodal'
output_dir='/home/ubuntu/weights/' + trained_model_name

# Initialize a new CustomViT model
model_name = "facebook/vit-mae-base"
vit_config = ViTMAEConfig.from_pretrained(model_name)
vit_config.output_hidden_states=True
vit_model = CustomViT(config=vit_config)

# Initialize a new CustomViTMAE model
model_name = "facebook/vit-mae-base"
config = ViTMAEConfig.from_pretrained(model_name)
config.output_hidden_states=True
custom_model = CustomViTMAE(config=config)
custom_model.vit = vit_model

# Load the state_dict from the saved model
state_dict = torch.load(f"{output_dir}/pytorch_model.bin")
custom_model.load_state_dict(state_dict)

# don't need decoders
vit_encoder = custom_model.vit

In [2]:
from tem_dataloader import MultimodalDataset
import functools

environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy'
dataset = MultimodalDataset(environemnt_directory)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P001
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P004
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P006
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P005
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P008
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P002
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P009
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P003
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P000
Number of images: 8185
Number 

In [9]:
from tem_dataloader import MultimodalDatasetPerTrajectory
import functools
import os
from torch.utils.data import Dataset, DataLoader

environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy'
OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

for i in range(0,10):
    if i==7:
        continue
    trajectory_folder_path = os.path.join(environemnt_directory, f'P00{i}')
    my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
    train_dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Initialize empty arrays for observations, actions, rewards, and terminals
    all_observations = np.empty((0, OBSERVATION_SIZE))
    all_actions = np.empty((0, ACTION_SIZE))
    all_rewards = np.empty(0)
    all_terminals = np.empty(0, dtype=bool)

    for batch_idx, data in enumerate(train_dataloader):
        # get embedding
        vit_encoder.cuda()
        vit_encoder.eval()
        pixel_values = data["pixel_values"].cuda()
        pixel_values1 = data["pixel_values1"].cuda()
        pixel_values2 = data["pixel_values2"].cuda()
        outputs = vit_encoder(pixel_values,pixel_values1,pixel_values2,noise=None)
        embedding = outputs.last_hidden_state[:,0,:]
        observation = embedding.cpu().detach().numpy()
        # get action
        pose = data["pose_values"]
        action = torch.diff(pose,axis = 0).numpy()
        action = np.concatenate((action, np.zeros((1,7))), axis=0)
        # get reward
        goal = observation[-1]
        partial_function = functools.partial(reward_function, goal_embedding=goal)
        reward = np.apply_along_axis(partial_function, 1, observation)
        # get terminals
        terminals = np.zeros_like(reward, dtype=int)
        terminals[reward == 100] = 1

        # Concatenate observations, actions, rewards, and terminals
        all_observations = np.vstack((all_observations, observation))
        all_actions = np.vstack((all_actions, action))
        all_rewards = np.hstack((all_rewards, reward))
        all_terminals = np.hstack((all_terminals, terminals))

    print("All observations shape:", all_observations.shape)
    print("All actions shape:", all_actions.shape)
    print("All rewards shape:", all_rewards.shape)
    print("All terminals shape:", all_terminals.shape)
    
    np.save(f'trajectories/all_observations_P00{i}.npy', all_observations)
    np.save(f'trajectories/all_actions_P00{i}.npy', all_actions)
    np.save(f'trajectories/all_rewards_P00{i}.npy', all_rewards)
    np.save(f'trajectories/all_terminals_P00{i}.npy', all_terminals)

Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P000
Number of images: 1480
Number of depth: 1480
Number of lidar: 1480
Number of pose: 1480
All observations shape: (8, 768)
All actions shape: (8, 7)
All rewards shape: (8,)
All terminals shape: (8,)
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P001
Number of images: 1055
Number of depth: 1055
Number of lidar: 1055
Number of pose: 1055
All observations shape: (31, 768)
All actions shape: (31, 7)
All rewards shape: (31,)
All terminals shape: (31,)
Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P002
Number of images: 698
Number of depth: 698
Number of lidar: 698
Number of pose: 698


KeyboardInterrupt: 

In [22]:
from tem_dataloader import MultimodalDatasetPerTrajectory
# environment_name = 'AmericanDinerExposure'
# environemnt_directory = f'/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/{environment_name}/Data_easy'
# my_dataset = MultimodalDatasetPerTrajectory(environemnt_directory)
environment_name = 'AbandonedFactoryExposure'
environemnt_directory = f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy'
OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

# for folder in os.listdir(environemnt_directory):
trajectory_folder_path = os.path.join(environemnt_directory, 'P000')
my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
train_dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=False)

Processing folder: /mnt/data/tartanairv2filtered/AbandonedFactoryExposure/Data_easy/P000
Number of images: 1480
Number of depth: 1480
Number of lidar: 1480
Number of pose: 1480


In [4]:
import numpy as np

def reward_function(state_embedding, goal_embedding, threshold=0.05, goal_reward=100):
    distance = np.linalg.norm(state_embedding - goal_embedding)

    if distance <= threshold:
        # Give a large positive reward when the goal is reached
        reward = goal_reward
    else:
        # Give a negative reward proportional to the distance otherwise
        reward = -distance

    return reward

In [24]:
import functools
import numpy as np
import torch

# Initialize empty arrays for observations, actions, rewards, and terminals
all_observations = np.empty((0, OBSERVATION_SIZE))
all_actions = np.empty((0, ACTION_SIZE))
all_rewards = np.empty(0)
all_terminals = np.empty(0, dtype=bool)

for batch_idx, data in enumerate(train_dataloader):
    # get embedding
    vit_encoder.cuda()
    vit_encoder.eval()
    pixel_values = data["pixel_values"].cuda()
    pixel_values1 = data["pixel_values1"].cuda()
    pixel_values2 = data["pixel_values2"].cuda()
    outputs = vit_encoder(pixel_values,pixel_values1,pixel_values2,noise=None)
    embedding = outputs.last_hidden_state[:,0,:]
    observation = embedding.cpu().detach().numpy()
    # get action
    pose = data["pose_values"]
    action = torch.diff(pose,axis = 0).numpy()
    action = np.concatenate((action, np.zeros((1,7))), axis=0)
    # get reward
    goal = observation[-1]
    partial_function = functools.partial(reward_function, goal_embedding=goal)
    reward = np.apply_along_axis(partial_function, 1, observation)
    # get terminals
    terminals = [False]*BATCH_SIZE
    terminals[-1]=True
    terminals = np.array(terminals)
    if batch_idx==0:
        print(observation.shape)
        print(action.shape)
        print(reward.shape)
        print(terminals.shape)

(64, 768)
(64, 7)
(64,)
(64,)


In [25]:
print("All observations shape:", all_observations.shape)
print("All actions shape:", all_actions.shape)
print("All rewards shape:", all_rewards.shape)
print("All terminals shape:", all_terminals.shape)

All observations shape: (1480, 768)
All actions shape: (1480, 7)
All rewards shape: (1480,)
All terminals shape: (1536,)


In [26]:
np.save('all_observations_P000.npy', all_observations)
np.save('all_actions_P000.npy', all_actions)
np.save('all_rewards_P000.npy', all_rewards)
np.save('all_terminals_P000.npy', all_terminals)

In [15]:
'''
Args:
        observations (numpy.ndarray): N-D array. If the
            observation is a vector, the shape should be
            `(N, dim_observation)`. If the observations is an image, the shape
            should be `(N, C, H, W)`.
        actions (numpy.ndarray): N-D array. If the actions-space is
            continuous, the shape should be `(N, dim_action)`. If the
            action-space is discrete, the shape should be `(N,)`.
        rewards (numpy.ndarray): array of scalar rewards. The reward function
            should be defined as :math:`r_t = r(s_t, a_t)`.
        terminals (numpy.ndarray): array of binary terminal flags.
        episode_terminals (numpy.ndarray): array of binary episode terminal
            flags. The given data will be splitted based on this flag.
            This is useful if you want to specify the non-environment
            terminations (e.g. timeout). If ``None``, the episode terminations
            match the environment terminations.
        discrete_action (bool): flag to use the given actions as discrete
            action-space actions. If ``None``, the action type is automatically
            determined.
    '''
cql_dataset = MDPDataset(observations=all_observations,actions=all_actions,rewards=all_rewards,terminals=all_terminals,episode_terminals=all_terminals)

In [18]:
from d3rlpy.algos import CQL

# setup CQL algorithm
cql = CQL(use_gpu=False)

# split train and test episodes
# train_episodes, test_episodes = train_test_split(cql_dataset, test_size=0.25)

# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=10,
        scorers=None)

2023-05-03 01:12:41 [debug    ] RoundIterator is selected.
2023-05-03 01:12:41 [info     ] Directory is created at d3rlpy_logs/CQL_20230503011241
2023-05-03 01:12:41 [debug    ] Building models...
2023-05-03 01:12:41 [debug    ] Models have been built.
2023-05-03 01:12:41 [info     ] Parameters are saved to d3rlpy_logs/CQL_20230503011241/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rat

Epoch 1/10:   0%|          | 0/4 [00:00<?, ?it/s]

2023-05-03 01:12:41 [info     ] CQL_20230503011241: epoch=1 step=4 epoch=1 metrics={'time_sample_batch': 0.0007401108741760254, 'time_algorithm_update': 0.13943541049957275, 'temp_loss': 11.497802257537842, 'temp': 0.9997500032186508, 'alpha_loss': -31.219698429107666, 'alpha': 1.0002500414848328, 'critic_loss': 578.8850479125977, 'actor_loss': -4.8624091148376465, 'time_step': 0.14036989212036133} step=4
2023-05-03 01:12:41 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230503011241/model_4.pt


Epoch 2/10:   0%|          | 0/4 [00:00<?, ?it/s]

2023-05-03 01:12:42 [info     ] CQL_20230503011241: epoch=2 step=8 epoch=2 metrics={'time_sample_batch': 0.0008065700531005859, 'time_algorithm_update': 0.13902419805526733, 'temp_loss': 11.686041593551636, 'temp': 0.9993497878313065, 'alpha_loss': -31.282936573028564, 'alpha': 1.0006502270698547, 'critic_loss': 575.037727355957, 'actor_loss': -5.53440248966217, 'time_step': 0.1400248408317566} step=8
2023-05-03 01:12:42 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230503011241/model_8.pt


...

Epoch 9/10:   0%|          | 0/4 [00:00<?, ?it/s]

2023-05-03 01:12:45 [info     ] CQL_20230503011241: epoch=9 step=36 epoch=9 metrics={'time_sample_batch': 0.0007060766220092773, 'time_algorithm_update': 0.09556132555007935, 'temp_loss': 11.737273693084717, 'temp': 0.9965471774339676, 'alpha_loss': -31.345335483551025, 'alpha': 1.0034571886062622, 'critic_loss': 571.6585922241211, 'actor_loss': -6.738372325897217, 'time_step': 0.0964612364768982} step=36
2023-05-03 01:12:45 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230503011241/model_36.pt


Epoch 10/10:   0%|          | 0/4 [00:00<?, ?it/s]

2023-05-03 01:12:45 [info     ] CQL_20230503011241: epoch=10 step=40 epoch=10 metrics={'time_sample_batch': 0.0007943511009216309, 'time_algorithm_update': 0.09351634979248047, 'temp_loss': 11.736368417739868, 'temp': 0.9961478263139725, 'alpha_loss': -31.333800792694092, 'alpha': 1.0038590133190155, 'critic_loss': 571.3820266723633, 'actor_loss': -6.799766898155212, 'time_step': 0.09450554847717285} step=40
2023-05-03 01:12:45 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20230503011241/model_40.pt


[(1,
  {'time_sample_batch': 0.0007401108741760254,
   'time_algorithm_update': 0.13943541049957275,
   'temp_loss': 11.497802257537842,
   'temp': 0.9997500032186508,
   'alpha_loss': -31.219698429107666,
   'alpha': 1.0002500414848328,
   'critic_loss': 578.8850479125977,
   'actor_loss': -4.8624091148376465,
   'time_step': 0.14036989212036133}),
 (2,
  {'time_sample_batch': 0.0008065700531005859,
   'time_algorithm_update': 0.13902419805526733,
   'temp_loss': 11.686041593551636,
   'temp': 0.9993497878313065,
   'alpha_loss': -31.282936573028564,
   'alpha': 1.0006502270698547,
   'critic_loss': 575.037727355957,
   'actor_loss': -5.53440248966217,
   'time_step': 0.1400248408317566}),
 (3,
  {'time_sample_batch': 0.0009879469871520996,
   'time_algorithm_update': 0.12402170896530151,
   'temp_loss': 11.76022744178772,
   'temp': 0.9989491403102875,
   'alpha_loss': -31.269726753234863,
   'alpha': 1.0010507106781006,
   'critic_loss': 574.1525115966797,
   'actor_loss': -6.174556

In [3]:
import numpy as np
from d3rlpy.dataset import Episode, MDPDataset, Transition

OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

all_observations = np.empty((0, OBSERVATION_SIZE))
all_actions = np.empty((0, ACTION_SIZE))
all_rewards = np.empty(0)
all_terminals = np.empty(0, dtype=bool)

for i in range(0,10):
    if i==7:
        continue
    observation = np.load(f'trajectories/all_observations_P00{i}.npy')
    action = np.load(f'trajectories/all_actions_P00{i}.npy')
    reward = np.load(f'trajectories/all_rewards_P00{i}.npy')
    terminals = np.load(f'trajectories/all_terminals_P00{i}.npy')

    all_observations = np.vstack((all_observations, observation))
    all_actions = np.vstack((all_actions, action))
    all_rewards = np.hstack((all_rewards, reward))
    all_terminals = np.hstack((all_terminals, terminals))

    print("All observations shape:", all_observations.shape)
    print("All actions shape:", all_actions.shape)
    print("All rewards shape:", all_rewards.shape)
    print("All terminals shape:", all_terminals.shape)
cql_dataset = MDPDataset(observations=all_observations,actions=all_actions,rewards=all_rewards,terminals=all_terminals,episode_terminals=all_terminals)

All observations shape: (1480, 768)
All actions shape: (1480, 7)
All rewards shape: (1480,)
All terminals shape: (1536,)
All observations shape: (2535, 768)
All actions shape: (2535, 7)
All rewards shape: (2535,)
All terminals shape: (2624,)
All observations shape: (2651, 768)
All actions shape: (2651, 7)
All rewards shape: (2651,)
All terminals shape: (2752,)
All observations shape: (2749, 768)
All actions shape: (2749, 7)
All rewards shape: (2749,)
All terminals shape: (2880,)
All observations shape: (2827, 768)
All actions shape: (2827, 7)
All rewards shape: (2827,)
All terminals shape: (3008,)
All observations shape: (2935, 768)
All actions shape: (2935, 7)
All rewards shape: (2935,)
All terminals shape: (3136,)
All observations shape: (3017, 768)
All actions shape: (3017, 7)
All rewards shape: (3017,)
All terminals shape: (3264,)
All observations shape: (3059, 768)
All actions shape: (3059, 7)
All rewards shape: (3059,)
All terminals shape: (3392,)
All observations shape: (3083, 7

In [4]:
from d3rlpy.algos import CQL
cql01 = CQL(use_gpu=False)
cql01.build_with_dataset(cql_dataset)
cql01.load_model('/home/ubuntu/camelmera/models/gym/multimodal/d3rlpy_logs/CQL_20230503011241/model_40.pt')